In [4]:
import chromadb
from datetime import datetime

# other function def
def getSecondsDiff(start,end):
    time_dif = end - start
    return 0.2 if time_dif.seconds ==0 else time_dif.seconds

def getTimeDiffString(time_dif_seconds) : 
    seconds = time_dif_seconds

    # 시간
    hours = seconds // 3600
    seconds %= 3600

    # 분
    minutes = seconds // 60
    seconds %= 60

    return f"[{int(hours)}h-{int(minutes)}m-{int(seconds)}s] left"

In [5]:
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

In [6]:
COLLECTION_NAME = "interview_data"

client = chromadb.PersistentClient()
collection = client.get_or_create_collection(name=COLLECTION_NAME)

In [ ]:
import pandas as pd
import gc
import torch
from datetime import datetime

# 데이터베이스에서 major_interview, new_major_professor 테이블 데이터 로드
import pymysql

password = 'YOUR_PASSWORD'
db_name = 'cnu_data'
user = 'USER_NAME'
# 데이터베이스 커넥션 생성
connection = pymysql.connect(
    host='localhost',
    port=3306,
    user=user,
    password=password,
    db=db_name,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

# major_interview 테이블 데이터 로드
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM major_interview")
    interview_data = cursor.fetchall()

# new_major_professor 테이블 데이터 로드
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM new_major_professor")
    professor_data = cursor.fetchall()

ids = []
metadatas = []
documents = []

start_time = datetime.now()
count = 0
total_count = len(interview_data)

# departments 테이블의 모든 데이터 미리 로드
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM departments")
    departments_data = cursor.fetchall()
departments_df = pd.DataFrame(departments_data)

# new_major_professor도 DataFrame으로 변환
professor_df = pd.DataFrame(professor_data)
interview_df = pd.DataFrame(interview_data)

from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_text(text, chunk_size=500, overlap=150):
    """
    langchain의 RecursiveCharacterTextSplitter를 사용하여
    텍스트를 chunk_size만큼 자르고, 각 청크는 overlap만큼 겹치게 반환
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_text(text)

for idx, row in interview_df.iterrows():
    question = row['question']
    answer = row['answer']
    answer_id = int(row['id'])
    professor_id = int(row['professor_id'])

    # 1. new_major_professor에서 professor_id와 일치하는 데이터 조회
    professor_row = professor_df[professor_df['id'] == professor_id]
    if professor_row.empty:
        department_value = None
    else:
        origin_department_id = professor_row.iloc[0]['origin_department_id']
        # 2. departments에서 origin_department_id와 일치하는 데이터 조회
        department_row = departments_df[departments_df['id'] == origin_department_id]
        if department_row.empty:
            department_value = None
        else:
            department_value = department_row.iloc[0]['department']


    if department_value is None:
        continue
    
    # 답변을 500자, 오버랩 150자로 청크 분할
    answer_chunks = chunk_text(str(answer), chunk_size=500, overlap=150)
    for chunk in answer_chunks:
        document = f"질문 : {question} 답변 : {chunk}"
        metadata = {
            "answer_id": answer_id,
            "professor_id": professor_id,
            "department": department_value
        }
        ids.append(f"{count}")
        documents.append(document)
        metadatas.append(metadata)
        count += 1

        # 배치 크기만큼 처리되면 DB에 추가
        if len(documents) >= 50:
            # 기존 데이터 존재 여부 확인
            existing_ids = collection.get(ids=ids)
            if existing_ids['ids'] and existing_ids['ids'][-1] in ids:
                ids = []
                metadatas = []
                documents = []
                if torch.backends.mps.is_available():
                    torch.mps.empty_cache()
                print(f"[debug] id = {existing_ids['ids'][-1]} 이전 데이터 건너뜀")
                continue

            embeddings = model.encode(documents)
            collection.add(
                ids=ids,
                metadatas=metadatas,
                documents=documents,
                embeddings=embeddings
            )
            end_time = datetime.now()
            prediction_seconds = getSecondsDiff(start_time, end_time) * total_count
            print("[debug] data added", end_time.strftime("%Y-%m-%d %H:%M:%S"),
                  getTimeDiffString(prediction_seconds))
            ids = []
            metadatas = []
            documents = []
            del embeddings
            gc.collect()
            if torch.backends.mps.is_available():
                torch.mps.empty_cache()

# 남은 데이터 처리
if len(documents) > 0:
    existing_ids = collection.get(ids=ids)
    if not (existing_ids['ids'] and existing_ids['ids'][-1] in ids):
        embeddings = model.encode(documents)
        collection.add(
            ids=ids,
            metadatas=metadatas,
            documents=documents,
            embeddings=embeddings
        )
        end_time = datetime.now()
        prediction_seconds = getSecondsDiff(start_time, end_time) * total_count
        print("[debug] data added", end_time.strftime("%Y-%m-%d %H:%M:%S"),
              getTimeDiffString(prediction_seconds))
        del embeddings
        gc.collect()
        if torch.backends.mps.is_available():
            torch.mps.empty_cache()
    ids = []
    metadatas = []
    documents = []


print("[debug] Vector db add complete")

[debug] data added 2025-07-26 03:27:41 [1h-11m-0s] left
[debug] data added 2025-07-26 03:27:42 [3h-33m-0s] left
[debug] data added 2025-07-26 03:27:44 [4h-44m-0s] left
[debug] data added 2025-07-26 03:27:45 [7h-6m-0s] left
[debug] data added 2025-07-26 03:27:46 [8h-17m-0s] left
[debug] data added 2025-07-26 03:27:47 [9h-28m-0s] left
[debug] data added 2025-07-26 03:27:49 [10h-39m-0s] left
[debug] data added 2025-07-26 03:27:50 [11h-50m-0s] left
[debug] data added 2025-07-26 03:27:51 [14h-12m-0s] left
[debug] data added 2025-07-26 03:27:52 [15h-23m-0s] left
[debug] data added 2025-07-26 03:27:53 [16h-34m-0s] left
[debug] data added 2025-07-26 03:27:55 [17h-45m-0s] left
[debug] data added 2025-07-26 03:27:56 [20h-7m-0s] left
[debug] data added 2025-07-26 03:27:57 [21h-18m-0s] left
[debug] data added 2025-07-26 03:27:58 [22h-29m-0s] left
[debug] data added 2025-07-26 03:28:00 [23h-40m-0s] left
[debug] data added 2025-07-26 03:28:01 [26h-2m-0s] left
[debug] data added 2025-07-26 03:28:02 [

In [13]:
# 컴퓨터공학과 관련 쿼리 검색
query = "나중에 사업을 좀 하고싶은데 어디로 가야하지"
results = collection.query(
    query_texts=[query],
    query_embeddings=[model.encode(query)],
    n_results=10
)

print("검색 결과:")
for idx, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
    print(f"\n--- 결과 {idx+1} ---")
    print(f"문서 내용: {doc}")
    print(f"메타데이터: {metadata}")


검색 결과:

--- 결과 1 ---
문서 내용: 질문 : 이 학과를 전공하려는 학생이 진학 전에 미리 준비해야 할 내용이 있다면 무엇입니까? 답변 : 준비할 것은 요즘 학생들은 보면은 미래에 대해서 하고 싶은 것이 없어요. 저희들이 제일 걱정하는 것이 그거에요. 뭐든지 내가 뭔가 하고 싶은 것이 있으면 그것이 추동력이 되어서 자기 인생을 잘 밀어 올릴 수 있는데 너무 편해서 그런 것 같아요. 부모님들이 너무 요즘에는 자식 하나 밖에 없어서 집에서 왕자처럼 자라다 보니까 하고 싶은 것이 없어요. 그것의 부작용이라고 저는 생각을 하는데 뭔가 내 삶에서 사회에서 또는 이 지구에서 뭔가 하고 싶은 기여하고 싶은 것들을 만들어 내지 않는 것이 가장 큰 문제라고 봐요. 학문적으로는 영어, 수학, 물리로 어떤 자기의 합리적인 사고 방식을 기르는 거죠. 방법론적으로 그런 것들이 잘 되어 있어야 우리가 탐구하려고 하는 대상에 대해서 효율적으로 접근할 수 있는 거에요. 그런 것들은 효율성과 관련된 거죠. 그런데 본질적으로 내 마음에서 그런 것을 추구하는 마음이 없으면 그런 것들이 다 소용이 없는 거죠.
메타데이터: {'answer_id': 905, 'professor_id': 73, 'department': '유기재료공학'}

--- 결과 2 ---
문서 내용: 질문 : 학과의 앞으로의 전망은 어떠하다고 보십니까? 답변 : 최근에 건축 붐이 그렇게 좋지는 않아요. 그래서 오히려 지금 최근의 건축의 방향은 우리나라 도시들이 노후화 되면서 재생, 도시재생이라든지 그 다음에 리모델링 그러면서 이제 아까 말씀 드렸던 친환경이나 에너지 절감형 건축물 쪽으로 많이 해서 전략적으로 갈 것 같아요.
메타데이터: {'professor_id': 49, 'answer_id': 601, 'department': '건축공학'}

--- 결과 3 ---
문서 내용: 질문 : 교수님께서는 이 학과(전공)를 선택하시게 된 동기는 무엇이었습니까? 답변 : 원래는 제가 컴퓨터 계열이었어요. 잘 아시다시피 공